In [9]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(__file__), ".."))

NameError: name '__file__' is not defined

In [42]:
import functools

def configure_gpu(world_conf):
    # the logic of world_conf follows "a,b,c,d,e" where:
    # the block range from 'a' to 'b' with interval 'c' (and each integer will repeat for 'd' time);
    # the block will be repeated for 'e' times.
    start, stop, interval, local_repeat, block_repeat = [
        int(x) for x in world_conf.split(",")
    ]
    _block = [
        [x] * local_repeat for x in range(start, stop + 1, interval)
    ] * block_repeat
    world_list = functools.reduce(lambda a, b: a + b, _block)
    return world_list

class PhysicalLayout(object):
    def __init__(self, n_participated, world, world_conf, on_cuda):
        self.n_participated = n_participated
        self._world = self.configure_world(world, world_conf)
        self._on_cuda = on_cuda
        self.rank = -1

    def configure_world(self, world, world_conf):
        if world is not None:
            world_list = world.split(",")
            assert self.n_participated <= len(world_list)
        elif world_conf is not None:
            # the logic of world_conf follows "a,b,c,d,e" where:
            # the block range from 'a' to 'b' with interval 'c' (and each integer will repeat for 'd' time);
            # the block will be repeated for 'e' times.
            return configure_gpu(world_conf)
        else:
            raise RuntimeError(
                "you should at least make sure world or world_conf is not None."
            )
        return [int(l) for l in world_list]

    @property
    def primary_device(self):
        return self.devices[0]

    @property
    def devices(self):
        return self.world

    @property
    def on_cuda(self):
        return self._on_cuda

    @property
    def ranks(self):
        return list(range(1 + self.n_participated))

    @property
    def world(self):
        return self._world

    def get_device(self, rank):
        return self.devices[rank]

    def change_n_participated(self, n_participated):
        self.n_participated = n_participated


def define_graph_topology(world, world_conf, n_participated, on_cuda):
    return PhysicalLayout(
        n_participated=n_participated,
        world=world,
        world_conf=world_conf,
        on_cuda=on_cuda,
    )


In [34]:
world_conf="0,0,1,1,100"

In [35]:
world=None

In [36]:
on_cuda=True

In [37]:
n_participated=5

In [38]:
graph =define_graph_topology(world_conf,world,n_participated,on_cuda)

In [39]:
graph.world

[0, 0, 1, 1, 100]

In [40]:
graph.get_device

<bound method PhysicalLayout.get_device of <__main__.PhysicalLayout object at 0x7f6d786eb850>>

In [43]:
configure_gpu(world_conf)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:



class Graph():
    def __init__(self, rank, n_processes ,world, on_cuda):
        self._rank = rank
        self._n_processes = n_processes
        self._world = world
        self._on_cuda = on_cuda
        
    @property
    def rank(self):
        return self._rank
    
    @property
    def n_processes(self):
        return self._n_processes
    
    @property
    def on_cuda(self):
        return self._on_cuda
    
    
    @property
    def world(self):
        return self._world


    
        